In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter, MultipleLocator


In [ ]:
import sys
from pathlib import Path

repo_root = Path().resolve().parents[0]
sys.path.append(str(repo_root))

print("Added to sys.path:", repo_root)

from utils.common_functions import save_or_show, load_df


In [ ]:
def publication_rc():
    plt.rcParams.update({
        "figure.figsize": (12, 6),
        "axes.titlesize": 14,
        "axes.labelsize": 12,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
        "legend.fontsize": 10,
        "lines.linewidth": 1.0,
        "axes.titleweight": "bold",
        "axes.labelweight": "bold",
        "legend.frameon": False,
        "font.family": "DejaVu Sans",
    })

In [ ]:
# 1) DEMAND: IER/Cambridge sheet
# ===============================
def load_engineering_demand_from_ods(
    path_ods: str,
    sheet_name: str = "1",
    soc_labels = (
        "21 Science, research, engineering and technology professionals",
        "31 Science, engineering and technology associate professionals",
    ),
    years=range(2024, 2031),
):
    """
    Returns a DataFrame with columns: Year, Demand (thousands)
    We treat 'demand' as projected headcount (employment) proxy by summing SOC 21 + 31.
    """
    # Load as-is; we'll detect header row containing year columns.
    wb = pd.read_excel(path_ods, sheet_name=sheet_name, engine="odf", header=None)

    # Find the header row (first row that contains a 4-digit year like '2024')
    header_row = None
    year_cols = None
    for r in range(min(30, len(wb))):  # scan first 30 rows
        row_vals = wb.iloc[r].astype(str).str.strip().tolist()
        candidates = {i: v for i, v in enumerate(row_vals) if v.isdigit() and len(v) == 4}
        if candidates:
            header_row = r
            break

    if header_row is None:
        raise ValueError("Could not detect a year header row in ODS Sheet '1'.")

    # Set header
    df = pd.read_excel(path_ods, sheet_name=sheet_name, engine="odf", header=header_row)
    # Clean column names
    df.columns = [str(c).strip() for c in df.columns]
    # Keep only the SOC name + numeric year columns
    # Guess the SOC name column: the first column usually has occupation label
    soc_col = df.columns[0]

    # Keep only year columns we need
    year_cols = [str(y) for y in years if str(y) in df.columns]
    if not year_cols:
        raise ValueError("No requested year columns found in ODS.")

    # Filter to SOC 21 and 31
    mask = df[soc_col].astype(str).str.strip().isin(soc_labels)
    d2 = df.loc[mask, [soc_col] + year_cols].copy()

    # Melt to long
    long = d2.melt(id_vars=[soc_col], value_vars=year_cols,
                   var_name="Year", value_name="Employment")
    # Coerce numbers
    long["Employment"] = pd.to_numeric(long["Employment"], errors="coerce")
    long["Year"] = long["Year"].astype(int)

    # Sum SOC 21+31 for each year
    by_year = (long.groupby("Year", as_index=False)["Employment"]
                    .sum()
                    .rename(columns={"Employment": "Demand"}))

    # Some sheets are in persons; your Chart 8 was in thousands.
    # If these are persons, convert to thousands for consistency.
    # We’ll detect magnitude; if median > 1e6, divide by 1000.
    if by_year["Demand"].median() > 1e6:
        by_year["Demand"] = by_year["Demand"] / 1_000.0  # to thousands

    return by_year  # Year, Demand


In [ ]:
# 2) SUPPLY: HESA Engineering qualifiers
# =======================================
def load_engineering_supply_from_hesa_csv(
    path_csv: str,
    subject_match=("Engineering and technology", "Engineering & technology", "Engineering"),
    years=range(2024, 2031),
    level_keys=("qualifier", "qualifiers"),
    year_keys=("academic year", "academic_year", "year"),
    subject_keys=("subject", "common subject area", "cah level 1", "cah_l1")
):
    """
    Returns a DataFrame with columns: Year, Supply (graduates, thousands)
    Tries to find rows for 'Engineering and technology' across common column namings.
    """
    df = pd.read_csv(path_csv)
    # Standardize columns
    df.columns = [str(c).strip().lower() for c in df.columns]

    # Guess columns
    def pick_first(candidates):
        for c in candidates:
            if c in df.columns:
                return c
        raise KeyError(f"Could not find any of {candidates} in HESA CSV columns:\n{df.columns}")

    col_year = pick_first(year_keys)
    col_subject = pick_first(subject_keys)

    # If there is a 'measure' column (e.g., 'Qualifiers'), pick it; else try a numeric column.
    numeric_cols = [c for c in df.columns if df[c].dtype.kind in "fi"]
    if numeric_cols:
        col_value = numeric_cols[0]
    else:
        # Sometimes value is in a column named e.g. "value"
        val_candidates = ["value", "count", "number", "total"]
        col_value = pick_first(val_candidates)

    # Filter subject
    subj_mask = df[col_subject].astype(str).str.lower().str.contains(
        "|".join([s.lower() for s in subject_match]), na=False
    )
    d2 = df.loc[subj_mask, [col_year, col_subject, col_value]].copy()

    # Coerce year to int (extract 4-digit)
    d2["Year"] = d2[col_year].astype(str).str.extract(r"(20\d{2})")
    d2 = d2.dropna(subset=["Year"])
    d2["Year"] = d2["Year"].astype(int)

    # Keep required years
    d2 = d2[d2["Year"].isin(years)].copy()
    d2["Supply"] = pd.to_numeric(d2[col_value], errors="coerce")

    # Aggregate if multiple breakdowns exist (e.g., by domicile/level)
    d3 = (d2.groupby("Year", as_index=False)["Supply"].sum())

    # Convert to thousands to align with demand (if necessary)
    if d3["Supply"].median() > 1e6:
        d3["Supply"] = d3["Supply"] / 1_000.0
    elif d3["Supply"].median() > 1e3 and d3["Supply"].median() < 1e6:
        # very likely already in units (not thousands). Convert to thousands for consistency.
        d3["Supply"] = d3["Supply"] / 1_000.0

    return d3[["Year", "Supply"]]

In [ ]:
# 3) Plot dual line / area
# ===========================
def plot_demand_vs_supply(
    df_demand, df_supply,
    title="Projected Engineering Demand vs Graduate Supply (2024–2030)",
    ylabel="People (thousands)",
    out_basename="chart9_demand_vs_supply",
    chart_dir="chart-9",
    kind="line"  # "line" or "area"
):
    # Merge on Year
    merged = pd.merge(df_demand, df_supply, on="Year", how="inner").sort_values("Year")
    years = merged["Year"].tolist()
    demand = merged["Demand"].values
    supply = merged["Supply"].values

    # Size so legend fits on the right without wrapping
    nx = len(years)
    fig_w = max(12, 0.9 * nx + 8)  # wider than usual to avoid legend wrapping
    fig, ax = plt.subplots(figsize=(fig_w, 6))

    if kind == "area":
        ax.fill_between(years, demand, step="mid", alpha=0.3, label="Projected demand (employment proxy)")
        ax.fill_between(years, supply, step="mid", alpha=0.3, label="Graduate supply (HESA)")
        ax.plot(years, demand, linewidth=1.8)
        ax.plot(years, supply, linewidth=1.8)
    else:
        ax.plot(years, demand, linewidth=1.8, label="Projected demand (employment proxy)")
        ax.plot(years, supply, linewidth=1.8, label="Graduate supply (HESA)")

    # Gap annotation (optional): show demand - supply each year
    gap = demand - supply
    # ax2 = ax.twinx()
    # ax2.bar(years, gap, alpha=0.1)
    # ax2.set_ylim(bottom=min(0, gap.min())*1.1, top=max(0, gap.max())*1.1)

    ax.set_title(title, pad=14, fontweight="bold")
    ax.set_xlabel("Year")
    ax.set_ylabel(ylabel)
    ax.yaxis.set_major_formatter(StrMethodFormatter("{x:,.0f}"))
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.grid(True, axis="y", linestyle="--", alpha=0.4)

    # Legend outside, single row
    ax.legend(title="", loc="upper left", bbox_to_anchor=(1.02, 1), frameon=False)

    plt.tight_layout()

    # Use your save_or_show if available
    try:
        from utils.common_functions import save_or_show
        save_or_show(fig, out_dir=chart_dir, filename=out_basename, show=True)
    except Exception:
        plt.show()



In [ ]:
# 4) Run it
# ===========================
# File paths (adjust to your paths/names if needed)
ods_path = "../data/item-9/United_Kingdom_Additional_Tables.ods"
csv_path = "../data/item-9/table-43.csv"

# Load demand (IER/Cambridge) and supply (HESA)
years = range(2024, 2031)
df_demand = load_engineering_demand_from_ods(ods_path, years=years)
df_supply = load_engineering_supply_from_hesa_csv(csv_path, years=years)

# Plot (dual line by default; change kind="area" for filled areas)
plot_demand_vs_supply(
    df_demand,
    df_supply,
    title="Projected Engineering Job Openings vs Graduate Supply (2024–2030)",
    ylabel="People (thousands)",
    out_basename="projected_openings_vs_graduate_supply",
    chart_dir="chart-9",
    kind="line"
)